<a href="https://colab.research.google.com/github/lucasabbruzzini/Portfolio/blob/main/macro_indicators_correlation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup



## Initializing functions

In [ ]:
%%capture
!pip install fredapi
from fredapi import Fred
fred = Fred(api_key='d8454def6d3dc15ac928c365ec12b247') #add your fred api here

!pip install yfinance
import yfinance as yfin
from IPython.display import JSON
import json
import pandas as pd
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, RangeTool, WheelZoomTool, LegendItem, Range1d, HoverTool, LinearAxis, BasicTicker, ColorBar, LinearColorMapper, PrintfTickFormatter, LabelSet
from bokeh.plotting import figure, show
from bokeh.io import output_notebook,  push_notebook
output_notebook()

import itertools
from bokeh.palettes import Inferno as palette


## Base Functions

In [ ]:
def compare(main_indicator, comparison_indicators, data):
  df = data[main_indicator].df

  df.rename(columns = {0: data[main_indicator].ylabel})

  for i in comparison_indicators:
    data[i].df.rename(columns = {data[i].ylabel: f'{data[i].ylabel}-{data[i].id}'}, inplace = True)
    df = pd.merge(df, data[i].df, left_index=True, right_index=True, how="outer")



  df = df.resample('Q').mean()
  df = df.dropna()

  pearson = df.corr('pearson')
  spearman = df.corr('spearman')


  return df, pearson, spearman

def correlation(data):
  data.columns.name = 'column_names'
  data = data.rename_axis('index_names', axis='index')

  xlabel = list(data.index)
  ylabel = list(data.columns)
  df = pd.DataFrame(data.stack(), columns=['value']).reset_index()

  source= ColumnDataSource(round(df,3))

  colors = ["#75968f", "#a5bab7", "#c9d9d3", "#e2e2e2", "#dfccce", "#ddb7b1", "#cc7878", "#933b41", "#550b1d"]
  mapper = LinearColorMapper(palette=colors, low=1, high=-1)

  p = figure(title="Correlation Heatmap",
            x_range=xlabel, y_range=list(reversed(ylabel)),
            x_axis_location="below", width=1000, height=500)

  p.grid.grid_line_color = None
  p.axis.axis_line_color = None
  p.axis.major_tick_line_color = None
  p.axis.major_label_text_font_size = "10px"
  p.axis.major_label_standoff = 0
  p.xaxis.major_label_orientation = 0.5

  p.rect(x='index_names', y='column_names', width=1, height=1,
        source=source,
        fill_color={'field': 'value', 'transform': mapper},
        line_color=None)



  color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="10px",
                      ticker=BasicTicker(desired_num_ticks=(len(colors))-4),
                      border_line_color=None)
  p.add_layout(color_bar, 'right')


  labels = LabelSet(
      x='index_names',
      y='column_names',
      text = 'value',
      text_font_size = '10px',
      text_align='center',
      text_color = 'navy',
      source = source
  )

  p.add_layout(labels)
  print('\n')
  show(p)

def show_multiple(*args):

  colors = list(itertools.chain.from_iterable(map(lambda x: x, palette.values())))
  p = figure(height=500, width=1000, tools="xpan",
            x_axis_type="datetime", x_axis_location="below",
            background_fill_color="#efefef",  y_axis_label=args[0].ylabel)

  source = ColumnDataSource(data=args[0].df)
  line = p.line('Date', args[0].ylabel, source=source, color = colors[0], legend_label=args[0].ylabel)

  for i, data in enumerate(args[1:]):
    source = ColumnDataSource(data=data.df)
    name = f'extra_range_{i}'
    label = data.ylabel

    try:
      ymax = max(source.data[label])
      ymin = min(source.data[label])

    except:
      label = f'{data.ylabel}-{data.id}'
      ymax = max(source.data[label])
      ymin = min(source.data[label])

    p.line('Date', label, source=source, color=colors[i+1], y_range_name=name, legend_label=label)
    p.extra_y_ranges[name] = Range1d(ymin,ymax)
    p.add_layout(LinearAxis(axis_label=label, y_range_name=name), 'left')

  p.legend.location = 'top_left'
  p.legend.click_policy="mute"
  p.add_tools(HoverTool(), WheelZoomTool())
  show(p)

## Creating Data Class and methods

In [ ]:
class Data:

  def __init__(self, title, ylabel, id, source):

      self.title = title
      self.ylabel = ylabel
      self.id = id
      self.source = source
      self.df = self.get_data()

  def get_data(self):
      if self.source == 'fed':
        data = fred.get_series(self.id)
        df = pd.DataFrame(data, columns=[self.ylabel])

      elif self.source == 'yfinance':
        df = yfin.download(self.id)
        df = df.rename(columns={'Adj Close': self.ylabel})
        df = df.rename(columns={'Date': 'Date'})
        df = df.drop(columns={'High','Low','Open','Close','Volume'})

      df[self.ylabel] = pd.to_numeric(df[self.ylabel], errors='coerce')
      df.rename_axis('Date', axis='index', inplace=True)
      df = df.dropna()

      return df


  def show(self):
    source = ColumnDataSource(data=self.df)

    p = figure(height=500, width=1000, tools="xpan",
              x_axis_type="datetime", x_axis_location="below", title = self.title,
              background_fill_color="#efefef")

    p.line('Date', self.ylabel, source=source)
    p.yaxis.axis_label = self.ylabel
    p.add_tools(HoverTool(), WheelZoomTool())
    show(p)




# Data

## Data sample

In [ ]:
data_list = [
        ['unemployment','USA Civilian Unemployment Rate','Unemployment Rate','UNRATE','fed'],
        ['cc_interest_rates','Credit Card Average Interest Rate','Interest Rate','TERMCBCCALLNS','fed'],
        ['fed_rate','Federal Funds Effective Rate','Interest Rate','FEDFUNDS','fed'],
        ['cip','Consumer Price Index','Consumer Price Index','CPIAUCSL','fed'],
        ['delinquency_cc','Delinquency Rate on Credit Card Loans','Delinquency Rate','DRCCLACBS','fed'],
        ['delinquency_home','Delinquency Rate on Single-Family Residential Mortgages','Delinquency Rate','DRSFRMACBS','fed'],
        ['apple','Apple Inc. (AAPL)','Price (USD)','AAPL','yfinance'],
        ['house_price','House Price Index - Cumulative','House Price Index','USSTHPI','fed'],
        ['auto_price','Consumer Price Index Used Cars and Trucks','Consumer Price Index','CUSR0000SETA02','fed'],
        ['30y_mortgage','30-Year Fixed Rate Mortgage','Interest Rate','MORTGAGE30US','fed'],
        ['48m_auto','Finance Rate on Autos 48 Month Loan','Interest Rate','TERMCBAUTO48NS','fed'],
        ['delinquency_realstate','Delinquency Rate on Commercial Real Estate Loans','Delinquency Rate','DRCRELEXFACBS','fed'],
        ['realstate_loan',' Commercial Real Estate Loans','Amount in USD','CLDACBW027SBOG','fed'],
        ['sp500','S&P 500','Price (USD)','^GSPC','yfinance'],
        ['vix','CBOE Volatility Index (^VIX)','Volatility Index (^VIX)','^VIX','yfinance'],
        ['oil','Crude Oil Price(USD)','Price(USD)','CL=F','yfinance'],
        ['bond','Vanguard Intermediate-Term Corporate Bond Index Fund (VCIT)','Price (USD)','VCIT','yfinance'],
        ['2y_treasury','USA 2-Year Treasury Constant Maturity Rate','Interest Rate','DGS2','fed'],
        ['10y_treasury','USA 10-Year Treasury Constant Maturity Rate','Interest Rate','DGS10','fed'],
        ['30y_treasury','USA 30-Year Treasury Constant Maturity Rate','Interest Rate','DGS30','fed'],
        ['wulf','TeraWulf Inc. (WULF)','Price (USD)','WULF','yfinance'],
        ]


In [ ]:
#Emulating json

data_dict = {'unemployment': {'title': 'USA Civilian Unemployment Rate',
  'ylabel': 'Unemployment Rate',
  'id': 'UNRATE',
  'source': 'fed'},
 'cc_interest_rates': {'title': 'Credit Card Average Interest Rate',
  'ylabel': 'Interest Rate',
  'id': 'TERMCBCCALLNS',
  'source': 'fed'},
 'fed_rate': {'title': 'Federal Funds Effective Rate',
  'ylabel': 'Interest Rate',
  'id': 'FEDFUNDS',
  'source': 'fed'},
 'cip': {'title': 'Consumer Price Index',
  'ylabel': 'Consumer Price Index',
  'id': 'CPIAUCSL',
  'source': 'fed'},
 'delinquency_cc': {'title': 'Delinquency Rate on Credit Card Loans',
  'ylabel': 'Delinquency Rate',
  'id': 'DRCCLACBS',
  'source': 'fed'},
 'delinquency_home': {'title': 'Delinquency Rate on Single-Family Residential Mortgages',
  'ylabel': 'Delinquency Rate',
  'id': 'DRSFRMACBS',
  'source': 'fed'},
 'apple': {'title': 'Apple Inc. (AAPL)',
  'ylabel': 'Price (USD)',
  'id': 'AAPL',
  'source': 'yfinance'},
 'house_price': {'title': 'House Price Index - Cumulative',
  'ylabel': 'House Price Index',
  'id': 'USSTHPI',
  'source': 'fed'},
 'auto_price': {'title': 'Consumer Price Index Used Cars and Trucks',
  'ylabel': 'Consumer Price Index',
  'id': 'CUSR0000SETA02',
  'source': 'fed'},
 '30y_mortgage': {'title': '30-Year Fixed Rate Mortgage',
  'ylabel': 'Interest Rate',
  'id': 'MORTGAGE30US',
  'source': 'fed'},
 '48m_auto': {'title': 'Finance Rate on Autos 48 Month Loan',
  'ylabel': 'Interest Rate',
  'id': 'TERMCBAUTO48NS',
  'source': 'fed'},
 'delinquency_realstate': {'title': 'Delinquency Rate on Commercial Real Estate Loans',
  'ylabel': 'Delinquency Rate',
  'id': 'DRCRELEXFACBS',
  'source': 'fed'},
 'realstate_loan': {'title': ' Commercial Real Estate Loans',
  'ylabel': 'Amount in USD',
  'id': 'CLDACBW027SBOG',
  'source': 'fed'},
 'sp500': {'title': 'S&P 500',
  'ylabel': 'Price (USD)',
  'id': '^GSPC',
  'source': 'yfinance'},
 'vix': {'title': 'CBOE Volatility Index (^VIX)',
  'ylabel': 'Volatility Index (^VIX)',
  'id': '^VIX',
  'source': 'yfinance'},
 'oil': {'title': 'Crude Oil Price(USD)',
  'ylabel': 'Price(USD)',
  'id': 'CL=F',
  'source': 'yfinance'},
 'bond': {'title': 'Vanguard Intermediate-Term Corporate Bond Index Fund (VCIT)',
  'ylabel': 'Price (USD)',
  'id': 'VCIT',
  'source': 'yfinance'},
 '2y_treasury': {'title': 'USA 2-Year Treasury Constant Maturity Rate',
  'ylabel': 'Interest Rate',
  'id': 'DGS2',
  'source': 'fed'},
 '10y_treasury': {'title': 'USA 10-Year Treasury Constant Maturity Rate',
  'ylabel': 'Interest Rate',
  'id': 'DGS10',
  'source': 'fed'},
 '30y_treasury': {'title': 'USA 30-Year Treasury Constant Maturity Rate',
  'ylabel': 'Interest Rate',
  'id': 'DGS30',
  'source': 'fed'},
 'wulf': {'title': 'TeraWulf Inc. (WULF)',
  'ylabel': 'Price (USD)',
  'id': 'WULF',
  'source': 'yfinance'}}

## Creating the data objects and pulling data

In [ ]:
%%capture
#for data_list
#data = {key: Data(title, ylabel, id, source) for key, title, ylabel, id, source in data_list}
#for data dict, json
data = {key: Data(value['title'], value['ylabel'], value['id'], value['source']) for key, value in data_dict.items()}


## View data using Bokeh

In [ ]:
data['apple'].show()

# Analyzing some data

## 1st Scenario we want to analyze what metrics have higher delinquency rates in credit cards in the USA.


We observed a strong negative correlation of the delinquency rate with the S&P500 and the Consumer Price Index, and a positive correlation with the unemployment and the Fed Funds Rate. This confirms that delinquency rate increases either when the economic situation is deteriorating (leading indicator S&P going down or unemployment going up), or when the interest rates are going up.

In [ ]:
main_indicator = 'delinquency_cc'
comparison_indicators = [
                        'unemployment',
                        'cc_interest_rates',
                        'fed_rate',
                        'sp500',
                        'cip',
                        'oil'
                        ]

df, pearson, spearman = compare(main_indicator, comparison_indicators, data)

correlation(pearson)

show_multiple(data['delinquency_cc'], data['cc_interest_rates'], data['fed_rate'])


## Scenario 2 - Delinquency Rate in Mortgages


We observe a high positive correlation of the Delinquency Rate with Unemployment and, surprisingly, the Crude Oil price; while a strong negative correlation with the Mortgage Rate and the Fed Funds Rate as expected.

In [ ]:
main_indicator = 'delinquency_home'
comparison_indicators = [
                        'unemployment',
                        'fed_rate',
                        'house_price',
                        '30y_mortgage',
                        'sp500',
                        'cip',
                        'oil'
                        ]

df, pearson, spearman = compare(main_indicator, comparison_indicators, data)

correlation(pearson)

show_multiple(data['delinquency_home'], data['fed_rate'], data['oil'], data['unemployment'])

## Publicly traded common stock

We use AAPL as our main indicator and found that AAPL has a high positive correlation with the S&P (as expected since it's one of the principal components of the index), and a high correlation with the Consumer Price Index. Surprisingly, AAPL has a very low correlation with the VIX.

In [ ]:
#We compare the Price and volume of the Specific Stock Selected (for example Apple Inc - AAPL) with:

main_indicator = 'apple'
comparison_indicators = [
                        'sp500',
                        'vix',
                        'fed_rate',
                        'cip',
                        'oil'
                        ]

df, pearson, spearman = compare(main_indicator, comparison_indicators, data)

correlation(pearson)

show_multiple(data['apple'], data['sp500'], data['fed_rate'])


#Publicly traded  treasury or corporate bond

We see that since 2010 the Bonds have a high positive correlation with the S&P. This is probably because coincides with a period of very low interest rates. In the past the correlation was negative.

In [ ]:

main_indicator = 'bond'
comparison_indicators = [
#                        'sp500_invbond',
                        'sp500',
                        'vix',
                        'fed_rate',
                        'cip',
                        'oil'
                        ]

df, pearson, spearman = compare(main_indicator, comparison_indicators, data)

correlation(pearson)

show_multiple(data['bond'], data['sp500'], data['fed_rate'])

In [ ]:
!pip install nbconvert


In [ ]:
!jupyter nbconvert --execute --to html "Macro_indicators_correlation.ipynb"

[NbConvertApp] Converting notebook Macro_indicators_correlation.ipynb to html
[NbConvertApp] ERROR | unhandled iopub msg: colab_request
[NbConvertApp] ERROR | unhandled iopub msg: colab_request
[NbConvertApp] ERROR | unhandled iopub msg: colab_request
[NbConvertApp] ERROR | unhandled iopub msg: colab_request
[NbConvertApp] ERROR | unhandled iopub msg: colab_request
[NbConvertApp] ERROR | unhandled iopub msg: colab_request
[NbConvertApp] Writing 20603486 bytes to Macro_indicators_correlation.html
